In [1]:
import sys
from torch.utils.data import random_split
sys.path.append("../")

In [2]:
from datasets import datasets
import pickle
import os
import pandas
from PIL import Image
import matplotlib.pyplot as plt
import numpy
import cv2
import torch

Loading training set information from the CSV file

In [3]:
train_info = pandas.read_csv("../data/raw_data/train.csv")

Loading images from folders

In [4]:
images = []

for path in os.listdir("../data/raw_data/train_images"):
    img_file = os.path.join("../data/raw_data/train_images", path)
    images.append(img_file)

train_info['image'] = images

Representing number of images for each individual class

In [5]:
train_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19906 entries, 0 to 19905
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      19906 non-null  object
 1   Class   19906 non-null  object
 2   image   19906 non-null  object
dtypes: object(3)
memory usage: 466.7+ KB


In [6]:
train_info.rename(
    columns={
        "Class": "class",
        "ID": "id"
    }, inplace=True
)

In [7]:
train_info['class'].value_counts()

class
MIDDLE    10804
YOUNG      6706
OLD        2396
Name: count, dtype: int64

In [8]:
train_info['class'] = train_info['class'].map(
    {
        'YOUNG': 0,
        'MIDDLE': 1,
        'OLD': 2
    }
)

In [9]:
train_info['class']

0        1
1        0
2        1
3        0
4        1
        ..
19901    1
19902    0
19903    1
19904    1
19905    1
Name: class, Length: 19906, dtype: int64

In [10]:
%%time

def to_png(img):
    success, png_data = cv2.imencode('.png', numpy.array(img))
    if success == True:
        png_image = cv2.imdecode(png_data, cv2.IMREAD_UNCHANGED)
        return Image.fromarray(png_image)
    else:
        print('sdffdss')
        raise RuntimeError('conversion failed')
        
def to_rgb(img):
    file = Image.open(img)
    if numpy.array(file).shape[2] == 4:
        file = cv2.cvtColor(numpy.array(file), cv2.COLOR_BGRA2BGR)
        file = Image.fromarray(file)
    return to_png(file)

imgs = train_info['image']
train_info['image'] = imgs.apply(lambda img: to_rgb(img))

CPU times: user 22.4 s, sys: 799 ms, total: 23.2 s
Wall time: 25.2 s


In [11]:
train_info['image']

0        <PIL.Image.Image image mode=RGB size=53x77 at ...
1        <PIL.Image.Image image mode=RGB size=34x56 at ...
2        <PIL.Image.Image image mode=RGB size=64x82 at ...
3        <PIL.Image.Image image mode=RGB size=57x73 at ...
4        <PIL.Image.Image image mode=RGB size=37x76 at ...
                               ...                        
19901    <PIL.Image.Image image mode=RGB size=46x58 at ...
19902    <PIL.Image.Image image mode=RGB size=159x156 a...
19903    <PIL.Image.Image image mode=RGB size=29x39 at ...
19904    <PIL.Image.Image image mode=RGB size=121x161 a...
19905    <PIL.Image.Image image mode=RGB size=38x41 at ...
Name: image, Length: 19906, dtype: object

Noise Analysis

In [12]:
def is_noisy(image, threshold: int = 10):
    scaled_img = cv2.cvtColor(numpy.array(image), cv2.IMREAD_GRAYSCALE)
    variance = cv2.Laplacian(src=scaled_img, threshold=threshold).var()
    return variance < threshold

def is_blurred(image, threshold: int = 10):
    return True

def compute_psf_score(image):
    pass

In [13]:
def visualize_noisy_images(noisy_images: numpy.ndarray):
    _, ax = plt.subplots()
    
    for col in enumerate(noisy_images.shape[0]):
        for row in range(noisy_images.shape[0]):
            ax[col, row].imshow(noisy_images)
    plt.show()

Removing image noise using smoothing filters

In [14]:
def apply_median_filter(image: numpy.ndarray, kernel_size: int):
    """
    Function applies standard non-linear median filter 
    to image for removing salt-and-papper noise from image
    """
    if len(image) == 0: return 
    filtered_img = cv2.medianBlur(
        src=image,
        ksize=kernel_size
    )
    return filtered_img

def apply_bilateral_filtering(
    image: numpy.ndarray, 
    kernel_size: int,
    sigma_space: int, 
    sigma_color: int
):
    """
    Function applies bilateral filter to given image
    """
    smoothed_img = cv2.bilateralFilter(
        src=image,
        sigmaColor=sigma_color,
        sigmaSpace=sigma_space,
        d=kernel_size,
    )
    return smoothed_img

Image Deconvolution using Wiener Filtering

In [15]:
from skimage import restoration

psf =  numpy.array([
    [0, -1, 0],
    [-1, 5,-1],
    [0, -1, 0]
]).astype(numpy.float32)

def restore_blurred_img(img: numpy.ndarray, psf):
    # Perform blind deconvolution
    psf, deconvolved = restoration.unsupervised_wiener(img, psf=psf)
    return psf, deconvolved

Forming Data Augmentation Instructions

In [16]:
HEIGHT = 512
WIDTH = 512

In [17]:
from torchvision import transforms 
from PIL import Image

train_transformations = [
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.Resize((HEIGHT, WIDTH), interpolation=Image.NEAREST),
    transforms.RandomRotation(degrees=25),
    transforms.RandomAdjustSharpness(sharpness_factor=1.3, p=0.5),
]

eval_transformations = [
    transforms.ToTensor(),
    transforms.Resize((HEIGHT, WIDTH), interpolation=Image.NEAREST),
]

Splitting dataset

In [18]:
from torch.utils.data import random_split

train_size = int(train_info.shape[0] * 0.7) # 70% of the data will be in training set
evaluation_size = train_info.shape[0] - train_size # rest of 30% will be in evaluation set

train_d, test_d = random_split(
    train_info, 
    [train_size, evaluation_size]
)

Handling class disbalance by defining class weights

In [19]:
CLASS_WEIGHTS = torch.tensor(
    [
        1.0,
        1.5,
        2.0
    ]
)

Splitting data

In [20]:
# training set
train_dataset = datasets.FaceRecognitionDataset(
    images=numpy.array(train_info['image'])[train_d.indices],
    labels=numpy.array(train_info['class'])[train_d.indices],
    transformations=train_transformations,
    weights=CLASS_WEIGHTS,
)

# evaludation dataset
test_dataset = datasets.FaceRecognitionDataset(
    images=numpy.array(train_info['image'])[test_d.indices],
    labels=numpy.array(train_info['class'])[test_d.indices],
    transformations=eval_transformations,
    weights=CLASS_WEIGHTS
)

In [21]:
middle = [img for idx, img in enumerate(train_dataset.images) if train_dataset.labels[idx] == "MIDDLE"]
young = [img for idx, img in enumerate(train_dataset.images) if train_dataset.labels[idx] == "YOUNG"]
old = [img for idx, img in enumerate(train_dataset.images) if train_dataset.labels[idx] == "OLD"]

In [22]:
print(len(middle))
print(len(young))
print(len(old))

0
0
0


Image Visualization

In [23]:
import random
def visualize_k_random_images(images, k):
    """
    Function visualizes k images, extracted from given source
    
    Args:
        images - typing.List[str] - array of images
        k - number of images
    """
    _, ax = plt.subplots(2, len(images) // k)
    k_random_images = random.choice(size=k, a=images)
    for col in range(ax.shape[0]):
        for row in range(ax.shape[0]):
            ax[col, row].imshow(k_random_images.pop())

Texture Analysis

In [24]:
def local_binary_patterns(img):
    pass 

def local_ternary_patterns(img):
    pass 

def local_phase_quantization(img):
    pass

def gabor_filtering(img):
    pass

Evaluating Transformed and Recovered Images

In [25]:
from skimage.metrics import structural_similarity as ssim

def ssim_score(orig_img, blur_img, channel_axis):
    # return ssim(orig_img, blur_img, channel_axis=channel_axis)
    return 1

def normalized_cross_correlation(signal1, signal2):
    """
    Function computes standard Normalized Cross Correlation
    for given old and modified versions of the same image 
    
    Args:
        old_img (numpy.ndarray) - numpy.array object of old image
        new_img (numpy.ndarray) - numpy.array object of the modified image
    """
    # Calculate means of the signals
    return 1

In [26]:
imgs = train_info['image'].to_numpy()

for idx, image in zip(train_d.indices, train_dataset.images):
    ssim = ssim_score(numpy.array(imgs[idx]), numpy.array(image), 2)
    cc = normalized_cross_correlation(numpy.array(imgs[idx]), numpy.array(image))
    print('ssim - %s; normalized cross correlation - %s;' % (ssim, cc))

ssim - 1; normalized cross correlation - 1;
ssim - 1; normalized cross correlation - 1;
ssim - 1; normalized cross correlation - 1;
ssim - 1; normalized cross correlation - 1;
ssim - 1; normalized cross correlation - 1;
ssim - 1; normalized cross correlation - 1;
ssim - 1; normalized cross correlation - 1;
ssim - 1; normalized cross correlation - 1;
ssim - 1; normalized cross correlation - 1;
ssim - 1; normalized cross correlation - 1;
ssim - 1; normalized cross correlation - 1;
ssim - 1; normalized cross correlation - 1;
ssim - 1; normalized cross correlation - 1;
ssim - 1; normalized cross correlation - 1;
ssim - 1; normalized cross correlation - 1;
ssim - 1; normalized cross correlation - 1;
ssim - 1; normalized cross correlation - 1;
ssim - 1; normalized cross correlation - 1;
ssim - 1; normalized cross correlation - 1;
ssim - 1; normalized cross correlation - 1;
ssim - 1; normalized cross correlation - 1;
ssim - 1; normalized cross correlation - 1;
ssim - 1; normalized cross corre

Initializing pandas datasets

Saving datasets to pickle format

In [27]:
pickle.dump(obj=train_dataset, file=open("../data/augmented_data/training_set.pkl", mode='wb'))
pickle.dump(obj=test_dataset, file=open("../data/augmented_data/testing_set.pkl", mode='wb'))